# 🔄 LangGraph Workshop: Mastering Loops & Cycles

Welcome! In this hands-on workshop, you'll learn the **most important pattern** in production AI agents: **loops**.

## Why Loops Matter

Think about how you solve complex problems:
- You don't get the perfect answer on the first try
- You review, critique, and improve your work
- You gather information from multiple sources
- You iterate until you're satisfied

**Agents should work the same way!**

### The Problem with Linear Workflows

Most tutorials show you this:
```
User Question → LLM → Answer → Done
```

But real-world agents need to:
- 🔄 **Self-improve**: Generate → Critique → Refine → Repeat
- ? **Research deeply**: Plan → Search → Evaluate → Search more
- 🤝 **Collaborate**: Agent A → Agent B → Discuss → Converge
- 🎯 **Meet quality standards**: Keep trying until good enough

### What You'll Build Today

By the end of this workshop, you'll have built **5 different loop patterns** from scratch, understanding when and how to use each one.

**Let's transform your agents from one-shot wonders to resilient, self-improving systems!** 🚀

## 🎯 Learning Objectives

By the end of this workshop, you will be able to:

### 1️⃣ Understand Loop Fundamentals
- **Create cycles** in your LangGraph by routing edges back to previous nodes
- **Design exit conditions** that prevent infinite loops (safety first!)
- **Track state evolution** across multiple iterations

### 2️⃣ Implement 5 Loop Patterns

You'll build working examples of:

| Pattern | What You'll Learn | Difficulty |
|---------|------------------|------------|
| **Part 1: Fixed Iteration** | Loop exactly N times | ⭐ Beginner |
| **Part 2: Quality-Driven** | Loop until quality threshold met (with scoring!) | ⭐⭐ Intermediate |
| **Part 3: Plan-Execution** | Loop through different searches (not same action!) | ⭐⭐ Intermediate |
| **Part 4: Streaming & Persistent** | Watch loops in real-time, resume later | ⭐⭐⭐ Advanced |
| **Part 5: Multi-Agent Debate** | Two agents collaborate in a loop | ⭐⭐⭐ Advanced |

### 3️⃣ Apply to Real Problems

After each pattern, you'll understand:
- ✅ **When** to use this pattern
- ✅ **Why** it's better than alternatives
- ✅ **How** to adapt it to your use case
- ⚠️ **Common pitfalls** to avoid

### 4️⃣ Build Production-Ready Skills
- Combine multiple loop patterns
- Add safety limits and error handling
- Monitor and debug loop behavior
- Optimize for cost and latency

---

**Ready? Let's start with the simplest loop pattern!** 👇

## Setup & Imports

## Prerequisites

Before starting this workshop, ensure you have:

1. **Python packages installed** (see `requirements.txt`):
   ```bash
   pip install langgraph langchain-google-genai langchain-community python-dotenv pydantic
   ```

2. **API Keys configured** in `.env` file:
   ```
   GOOGLE_API_KEY=your_gemini_api_key_here
   GOOGLE_CSE_ID=your_custom_search_engine_id_here
   ```

3. **Basic LangGraph knowledge**: Understanding of StateGraph, nodes, edges, and state management

**Note**: All examples use real LLM calls and APIs. No mock data!

In [20]:
# Core Python
import os
import sqlite3
import atexit
import time
import re
from pathlib import Path
from typing import Annotated, Dict, List, Optional, Literal, Union

from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangGraph primitives
from langgraph.graph import StateGraph, END, add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import interrupt

# LangChain
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

---

## 🔁 Part 1: Your First Loop — Fixed Iteration

**Goal**: Build an agent that loops exactly 3 times, adding more detail with each pass.

### The Challenge

You ask: *"What are LangGraph loops?"*

- **Iteration 1**: Brief 1-sentence answer
- **Iteration 2**: Add 1-2 more sentences with details
- **Iteration 3**: Expand even further

**This is progressive elaboration** — a common pattern in content generation!

### What You'll Learn

✅ How to create a cycle by routing a conditional edge back to a node  
✅ How to track iterations with state  
✅ How to use a counter as an exit condition  

**Let's build it!** 👇

In [15]:
class SimpleLoopState(BaseModel):
    query: str = ""
    answer: str = ""
    iteration: int = 0
    max_iterations: int = 3
    
    class Config:
        arbitrary_types_allowed = True


def improve_answer(state: SimpleLoopState):
    """Each iteration adds more detail to the answer."""
    iteration = state.iteration + 1
    
    if iteration == 1:
        prompt = f"Give a brief 1-sentence answer: {state.query}"
    else:
        prompt = f"Expand this answer with more detail:\n\nQuery: {state.query}\nCurrent: {state.answer}\n\nAdd 1-2 more sentences."
    
    answer = llm.invoke(prompt).content
    print(f"\n[Iteration {iteration}] Answer length: {len(answer)} chars")
    print(f"Preview: {answer[:80]}...")
    
    return {"answer": answer, "iteration": iteration}


def should_continue_simple(state: SimpleLoopState) -> str:
    """Loop until max iterations reached."""
    if state.iteration < state.max_iterations:
        print(f"  → Looping back (iteration {state.iteration}/{state.max_iterations})")
        return "continue"
    else:
        print(f"  → Done! Reached {state.max_iterations} iterations")
        return "end"


# Build the loop
simple_loop = StateGraph(SimpleLoopState)
simple_loop.add_node("improve", improve_answer)
simple_loop.set_entry_point("improve")

# THE LOOP: conditional edge that can route back to "improve"
simple_loop.add_conditional_edges(
    "improve",
    should_continue_simple,
    {
        "continue": "improve",  # 🔄 Loop back!
        "end": END
    }
)

simple_agent = simple_loop.compile()
print("✅ Simple loop agent compiled")

✅ Simple loop agent compiled


In [16]:
# Try it: Watch the agent loop 3 times
result = simple_agent.invoke(SimpleLoopState(query="What are LangGraph loops?"))
print(f"\n{'='*60}")
print("FINAL ANSWER:")
print(f"{'='*60}")
print(result['answer'])


[Iteration 1] Answer length: 164 chars
Preview: LangGraph loops are a mechanism for creating stateful, cyclical workflows in Lan...
  → Looping back (iteration 1/3)

[Iteration 2] Answer length: 411 chars
Preview: LangGraph loops are a mechanism for creating stateful, cyclical workflows in Lan...
  → Looping back (iteration 2/3)

[Iteration 2] Answer length: 411 chars
Preview: LangGraph loops are a mechanism for creating stateful, cyclical workflows in Lan...
  → Looping back (iteration 2/3)

[Iteration 3] Answer length: 734 chars
Preview: LangGraph loops are a mechanism for creating stateful, cyclical workflows in Lan...
  → Done! Reached 3 iterations

FINAL ANSWER:
LangGraph loops are a mechanism for creating stateful, cyclical workflows in LangGraph, allowing agents to iterate and refine their actions based on previous steps. This enables more complex and dynamic interactions than linear chains, as the graph can return to previous nodes based on conditions or agent decisions. They 

---

### ✅ Part 1 Complete: Understanding Fixed-Iteration Loops

**What You Just Built:**

```mermaid
graph LR
    A[improve_answer] --> B{iteration < max?}
    B -->|Yes: continue| A
    B -->|No: end| C[END]
```

### 🔑 Key Concepts

1. **The Loop Edge**: `"continue": "improve"` creates the cycle
   ```python
   simple_loop.add_conditional_edges(
       "improve",
       should_continue_simple,  # Routing function decides: loop or exit?
       {"continue": "improve", "end": END}  # ← The magic happens here!
   )
   ```

2. **State Tracking**: `iteration` counter increments each pass
   - Iteration 1: Brief answer
   - Iteration 2: Add details
   - Iteration 3: Expand further → **EXIT**

3. **Exit Condition**: `iteration >= max_iterations` breaks the loop
   - **Critical**: Without this, infinite loop! 🔥
   - Always have a safety limit!

### 💡 When to Use Fixed-Iteration Loops

✅ **Good for**:
- **Ensembling**: Generate 5 candidates, pick best
- **Progressive elaboration**: Build up answer iteratively (as shown)
- **Multi-round reasoning**: Chain of Thought across N steps
- **Predictable workflows**: When you know exactly how many passes you need

❌ **Not good for**:
- Quality-based work (you don't know how many tries it'll take)
- Dynamic scenarios (where N isn't known upfront)

### 🎯 Try This!

**Challenge**: Modify the code to loop 5 times instead of 3. Run it and see the answer grow!

---

**Next**: What if we don't know how many iterations we need? What if we want to loop **until quality is good enough**? Let's find out! 👉

---

## 📊 Part 2: Quality-Driven Loop — Self-Critique with Scoring

**The Problem with Part 1**: What if your first answer is already perfect? Why waste 2 more LLM calls?

**Better approach**: Loop **until quality threshold is met**!

### The Challenge

You ask: *"Explain photosynthesis in mathematical terms"*

The agent should:
1. **Generate** an answer
2. **Score it** 0-100 based on 5 criteria
3. **If score < 80**: Provide suggestions and regenerate
4. **If score ≥ 80**: Done! ✅

Watch the score improve: 65 → 78 → 85 → **Exit!**

### What You'll Learn

✅ Quality-based exit conditions (not just counters)  
✅ Numeric scoring for measurable progress  
✅ Self-critique patterns (LLM evaluating itself)  
✅ Carrying feedback between iterations  

**Let's build it!** 👇

In [24]:
class ReflectionState(BaseModel):
    query: str = ""
    draft: str = ""
    critique_text: str = ""  # Renamed from 'critique' to avoid node name conflict
    score: int = 0  # Score out of 100
    iteration: int = 0
    max_iterations: int = 5  # Reduced from 10 for learners
    score_threshold: int = 80  # Quality threshold
    
    class Config:
        arbitrary_types_allowed = True


def generate_draft(state: ReflectionState):
    """Generate or refine based on critique."""
    iteration = state.iteration + 1
    
    if iteration == 1:
        prompt = f"Answer this query in 2-3 sentences: {state.query}"
    else:
        prompt = f"""Query: {state.query}

Previous answer (Score: {state.score}/100): {state.draft}

Critique and suggestions: {state.critique_text}

Rewrite the answer addressing ALL the suggestions. Make specific improvements mentioned."""
    
    draft = llm.invoke(prompt).content
    print(f"\n[GENERATE iteration {iteration}]")
    print(f"  Draft: {draft[:1000]}...")
    
    return {"draft": draft, "iteration": iteration}


def critique_draft(state: ReflectionState):
    """Evaluate the draft quality with a score out of 100."""
    prompt = f"""Evaluate this answer and give it a score from 0-100.

Query: {state.query}
Answer: {state.draft}

Scoring criteria:
1. Directly addresses the query (20 points)
2. Provides specific examples or evidence (20 points)
3. Well-structured with clear logic (20 points)
4. Uses concrete details, not vague generalities (20 points)
5. Complete, covers all aspects (20 points)

Format your response EXACTLY as:
SCORE: [number from 0-100]
REASONING: [brief explanation of the score]
SUGGESTIONS: [2-3 specific improvements if score < 80, or "None" if score >= 80]"""
    
    critique_text = llm.invoke(prompt).content.strip()
    
    # Extract score from response
    import re
    score_match = re.search(r'SCORE:\s*(\d+)', critique_text)
    score = int(score_match.group(1)) if score_match else 0
    
    print(f"[CRITIQUE iteration {state.iteration}]")
    print(f"  Score: {score}/100")
    if score >= state.score_threshold:
        print(f"  ✅ Quality threshold met! (>= {state.score_threshold})")
    else:
        print(f"  ❌ Below threshold ({state.score_threshold}), needs improvement")
        # Extract suggestions
        suggestions_match = re.search(r'SUGGESTIONS:\s*(.+?)(?:\n|$)', critique_text, re.DOTALL)
        if suggestions_match:
            suggestions = suggestions_match.group(1).strip()[:2000]
            print(f"  Suggestions: {suggestions}...")
    
    return {"critique_text": critique_text, "score": score}


def route_by_quality(state: ReflectionState) -> str:
    """Loop if score below threshold AND under max iterations."""
    if state.score >= state.score_threshold:
        print(f"  → Finalizing (quality threshold met: {state.score}/{state.score_threshold})")
        return "done"
    elif state.iteration >= state.max_iterations:
        print(f"  → Finalizing (max iterations reached, final score: {state.score}/100)")
        return "done"
    else:
        print(f"  → Refining (attempt {state.iteration + 1}, current score: {state.score}/100)")
        return "refine"


# Build reflection loop
reflection_loop = StateGraph(ReflectionState)
reflection_loop.add_node("generate", generate_draft)  # Node name: "generate"
reflection_loop.add_node("critique", critique_draft)  # Node name: "critique" (no conflict now!)

reflection_loop.set_entry_point("generate")
reflection_loop.add_edge("generate", "critique")

# THE LOOP: route back to generate if quality is insufficient
reflection_loop.add_conditional_edges(
    "critique",
    route_by_quality,
    {
        "refine": "generate",  # 🔄 Loop back to improve
        "done": END
    }
)

reflection_agent = reflection_loop.compile()
print("✅ Reflection agent compiled")

✅ Reflection agent compiled


In [28]:
# Try it: Watch self-critique in action
query = "Is Trump good for economy?"
result = reflection_agent.invoke(ReflectionState(query=query))

print(f"\n{'='*70}")
print(f"FINAL ANSWER (after {result['iteration']} iteration(s)):")
print(f"Final Score: {result['score']}/100")
print(f"{'='*70}")
print(result['draft'])


[GENERATE iteration 1]
  Draft: Assessing Trump's economic impact is complex and debated. Supporters point to tax cuts and deregulation as drivers of growth, while critics cite increased national debt and trade policies that harmed certain sectors. Overall, the economy saw growth during his presidency, but attributing this solely to his policies is difficult due to various global and pre-existing factors....


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..


[CRITIQUE iteration 1]
  Score: 75/100
  ❌ Below threshold (80), needs improvement
  Suggestions: 1. Mention specific examples of tax cuts (e.g., the Tax Cuts and Jobs Act of 2017) and deregulation policies....
  → Refining (attempt 2, current score: 75/100)

[GENERATE iteration 2]
  Draft: Okay, here's a revised answer incorporating the suggestions:

**Revised Answer (Score: 90/100):**

Assessing Trump's economic impact is complex and debated. Supporters point to the Tax Cuts and Jobs Act of 2017, which significantly lowered corporate and individual income tax rates, and deregulation efforts as drivers of economic growth. For example, there were efforts to roll back environmental regulations. Critics, however, cite a substantial increase in the national debt and trade policies that harmed certain sectors. During his presidency (2017-2021), the U.S. saw an average annual GDP growth rate of around 2.5%. However, the national debt increased significantly, rising from approximately $19.9 

---

### ✅ Part 2 Complete: Quality-Driven Loops with Scoring

**What You Just Built:**

```mermaid
graph LR
    A[generate] --> B[critique + score]
    B --> C{score >= 80?}
    C -->|Yes| D[END]
    C -->|No| A
```

### 🔑 Key Innovation: Measurable Quality

Instead of vague "is it good enough?", we have **objective scoring**:

```python
# 5 criteria × 20 points each = 100
1. Directly addresses query (20 pts)
2. Specific examples/evidence (20 pts)  
3. Well-structured logic (20 pts)
4. Concrete details (20 pts)
5. Complete coverage (20 pts)
```

**Why this is powerful:**
- 📊 **Trackable**: Watch score improve each iteration (65 → 78 → 85)
- 🎯 **Objective**: No ambiguity about "good enough"
- 🔧 **Tunable**: Adjust threshold (70 for lenient, 90 for strict)
- 🐛 **Debuggable**: See exactly which criteria failed

### 💡 Exit Conditions: Two Ways Out

```python
if score >= 80:
    return "done"  # ✅ Quality met!
elif iteration >= 5:
    return "done"  # ⚠️ Max reached, accept what we have
else:
    return "refine"  # 🔄 Try again!
```

**Always have both**:
1. ✅ **Quality check** (desired exit)
2. ⚠️ **Max iterations** (safety exit to prevent infinite loops)

### 📈 Real Output Example

```
[GENERATE iteration 1]
  Draft: Photosynthesis converts light to energy...

[CRITIQUE iteration 1]
  Score: 65/100
  ❌ Below threshold, needs improvement
  Suggestions: Add equations, more specifics...

[GENERATE iteration 2]  
  Draft: Photosynthesis: 6CO₂ + 6H₂O + light → C₆H₁₂O₆ + 6O₂...

[CRITIQUE iteration 2]
  Score: 85/100
  ✅ Quality threshold met!

FINAL ANSWER (after 2 iterations)
```

### 🎯 When to Use Quality-Driven Loops

✅ **Perfect for**:
- **Self-refinement**: Polish answers until good enough
- **Code generation**: Compile → Fix errors → Repeat until works
- **Content creation**: Draft → Critique → Improve
- **Validation**: Check output meets standards

❌ **Not ideal for**:
- Tasks where quality is subjective
- When you need deterministic iteration count
- High-cost scenarios (could loop many times)

### 🧪 Experiments to Try

1. **Adjust threshold**: Change `score_threshold` to 70 (easier) or 90 (stricter)
2. **Modify criteria**: Add your own scoring criteria
3. **Different queries**: Try questions that naturally score high vs. low

---

**Next**: We've looped over the *same* action (improve answer). But what if we need to loop through a *PLAN* of different actions? 👉

---

## 🔍 Part 3: Plan-Execution Loop — Multi-Search Research

**The Big Idea**: Parts 1 & 2 looped over the *same action*. Real agents need to loop through a *PLAN* of *different actions*!

### The Challenge

You ask: *"What are the benefits and challenges of LangGraph?"*

A single search won't capture everything. Instead:
1. **Plan**: Break into 3 focused searches:
   - Search 1: "LangGraph benefits production AI"
   - Search 2: "LangGraph challenges limitations"
   - Search 3: "LangGraph vs alternatives comparison"
2. **Execute loop**: Search 1 → Search 2 → Search 3
3. **Synthesize**: Combine all results into comprehensive answer

### Why This Pattern Matters

🎯 **This is how real research agents work**:
- Not just one search
- Multiple perspectives
- Comprehensive coverage

### What You'll Learn

✅ Looping through a **list of different actions** (not same action)  
✅ Accumulating results across iterations  
✅ Early exit if sufficient coverage  
✅ Real API integration (Google Search)  

### ⚙️ Prerequisites

This part uses **real Google Search API**. You'll need:
- `GOOGLE_API_KEY` - [Get it here](https://developers.google.com/custom-search/v1/overview)
- `GOOGLE_CSE_ID` - Custom Search Engine ID

Add to your `.env` file!

**Let's build a real research agent!** 👇

In [ ]:
from langchain_community.utilities import GoogleSearchAPIWrapper

class ResearchLoopState(BaseModel):
    query: str = ""
    search_plan: List[str] = Field(default_factory=list)  # List of queries to execute
    current_search_idx: int = 0
    all_results: List[Dict] = Field(default_factory=list)  # Accumulated results
    answer: str = ""
    is_sufficient: bool = False
    max_searches: int = 3
    
    class Config:
        arbitrary_types_allowed = True


def plan_searches(state: ResearchLoopState):
    """Create a plan of multiple search queries to gather comprehensive info."""
    prompt = f"""You are a research planner. Break this question into 2-3 focused search queries that cover different angles.

Question: {state.query}

For each query:
- Focus on a specific aspect or perspective
- Use effective search keywords
- Avoid overlap between queries

Return ONLY a numbered list of search queries (no explanations)."""
    
    plan_text = llm.invoke(prompt).content.strip()
    
    # Extract queries from numbered list
    queries = re.findall(r'^\d+\.\s*(.+)$', plan_text, re.MULTILINE)
    queries = [q.strip().replace('"', '') for q in queries[:3]]  # Max 3
    
    print(f"\n[PLAN] Created search plan with {len(queries)} queries:")
    for i, q in enumerate(queries, 1):
        print(f"  {i}. {q}")
    
    return {
        "search_plan": queries,
        "current_search_idx": 0,
        "all_results": []
    }


def execute_search(state: ResearchLoopState):
    """Execute the current search in the plan."""
    if state.current_search_idx >= len(state.search_plan):
        print(f"[SEARCH] No more searches in plan")
        return {}
    
    current_query = state.search_plan[state.current_search_idx]
    search_num = state.current_search_idx + 1
    
    print(f"\n[SEARCH {search_num}/{len(state.search_plan)}] Query: {current_query}")
    
    search_engine = GoogleSearchAPIWrapper()
    raw_results = search_engine.results(current_query, num_results=3)
    
    # Extract and tag results with search context
    results = [
        {
            "search_query": current_query,
            "search_num": search_num,
            "title": r.get("title", "No title"),
            "snippet": r.get("snippet", r.get("description", "No snippet")),
            "link": r.get("link", r.get("url", ""))
        }
        for r in raw_results
    ]
    
    print(f"  ✅ Found {len(results)} results")
    for i, r in enumerate(results, 1):
        print(f"    {i}. {r['title'][:50]}...")
    
    # Accumulate results
    all_results = state.all_results + results
    next_idx = state.current_search_idx + 1
    
    return {"all_results": all_results, "current_search_idx": next_idx}


def evaluate_coverage(state: ResearchLoopState):
    """Check if we have enough diverse information."""
    if not state.all_results:
        print(f"[EVALUATE] No results yet")
        return {"is_sufficient": False}
    
    # Format results grouped by search
    results_by_search = {}
    for r in state.all_results:
        search_num = r['search_num']
        if search_num not in results_by_search:
            results_by_search[search_num] = []
        results_by_search[search_num].append(r)
    
    summary = "\n\n".join([
        f"Search {num}: {results[0]['search_query']}\n" + 
        "\n".join([f"  - {r['title']}: {r['snippet'][:60]}..." for r in results])
        for num, results in sorted(results_by_search.items())
    ])
    
    prompt = f"""Evaluate if these search results provide comprehensive coverage to answer the question.

Question: {state.query}

Results Summary:
{summary}

Total sources: {len(state.all_results)}
Searches completed: {state.current_search_idx}/{len(state.search_plan)}

Reply 'SUFFICIENT' if you have enough diverse information to provide a complete answer.
Reply 'INSUFFICIENT' if more searches from the plan would help."""
    
    evaluation = llm.invoke(prompt).content.strip()
    is_sufficient = "sufficient" in evaluation.lower() and "insufficient" not in evaluation.lower()
    
    print(f"\n[EVALUATE] Coverage check:")
    print(f"  Sources gathered: {len(state.all_results)}")
    print(f"  Searches done: {state.current_search_idx}/{len(state.search_plan)}")
    
    if is_sufficient:
        print(f"  ✅ Coverage sufficient!")
    else:
        print(f"  ❌ Need more searches: {evaluation[:80]}...")
    
    return {"is_sufficient": is_sufficient}


def synthesize_research(state: ResearchLoopState):
    """Synthesize answer from all gathered results."""
    # Group by search for context
    results_by_search = {}
    for r in state.all_results:
        search_num = r['search_num']
        if search_num not in results_by_search:
            results_by_search[search_num] = []
        results_by_search[search_num].append(r)
    
    results_text = "\n\n".join([
        f"**Search {num}: {results[0]['search_query']}**\n" +
        "\n".join([f"- {r['title']}\n  {r['snippet']}\n  Source: {r['link']}" for r in results])
        for num, results in sorted(results_by_search.items())
    ])
    
    prompt = f"""Synthesize a comprehensive answer using ALL search results provided.

Question: {state.query}

Research Results:
{results_text}

Provide a complete answer that:
1. Integrates information from multiple searches
2. Cites sources with [source: URL]
3. Acknowledges different perspectives if present
4. Is well-structured and clear"""
    
    answer = llm.invoke(prompt).content
    print(f"\n[SYNTHESIZE] Generated comprehensive answer ({len(answer)} chars)")
    
    return {"answer": answer}


def should_continue_searching(state: ResearchLoopState) -> str:
    """Loop through planned searches until sufficient coverage."""
    if state.is_sufficient:
        print(f"  → Synthesizing (sufficient coverage)")
        return "synthesize"
    elif state.current_search_idx >= len(state.search_plan):
        print(f"  → Synthesizing (all planned searches done)")
        return "synthesize"
    elif state.current_search_idx >= state.max_searches:
        print(f"  → Synthesizing (max searches reached)")
        return "synthesize"
    else:
        remaining = len(state.search_plan) - state.current_search_idx
        print(f"  → Continuing research ({remaining} search(es) remaining)")
        return "continue"


# Build research loop
research_loop = StateGraph(ResearchLoopState)
research_loop.add_node("plan", plan_searches)
research_loop.add_node("search", execute_search)
research_loop.add_node("evaluate", evaluate_coverage)
research_loop.add_node("synthesize", synthesize_research)

research_loop.set_entry_point("plan")
research_loop.add_edge("plan", "search")
research_loop.add_edge("search", "evaluate")

# THE LOOP: continue executing searches from the plan
research_loop.add_conditional_edges(
    "evaluate",
    should_continue_searching,
    {
        "continue": "search",      # 🔄 Loop back to next search
        "synthesize": "synthesize"
    }
)

research_loop.add_edge("synthesize", END)

research_agent = research_loop.compile()
print("✅ Research loop agent compiled")

✅ Research loop agent compiled


In [ ]:
# Try it: Watch the agent execute multi-search research
query = "What are the benefits and challenges of using LangGraph for production AI systems?"
result = research_agent.invoke(ResearchLoopState(query=query))

print(f"\n{'='*70}")
print(f"RESEARCH COMPLETE")
print(f"{'='*70}")
print(f"\nSearch plan executed:")
for i, q in enumerate(result['search_plan'], 1):
    print(f"  {i}. {q}")
print(f"\nTotal sources gathered: {len(result['all_results'])}")
print(f"\n{'='*70}")
print("FINAL ANSWER:")
print(f"{'='*70}")
print(f"\n{result['answer']}")


[PLAN] Created search plan with 3 queries:
  1. LangGraph production deployment benefits challenges scalability
  2. LangGraph agent orchestration real-world applications limitations
  3. LangGraph state management complex workflows debugging monitoring

[SEARCH 1/3] Query: LangGraph production deployment benefits challenges scalability
  ✅ Found 3 results
    1. LangGraph in production? : r/LangChain...
    2. LangGraph overview - Docs by LangChain...
    3. LangGraph...
  ✅ Found 3 results
    1. LangGraph in production? : r/LangChain...
    2. LangGraph overview - Docs by LangChain...
    3. LangGraph...

[EVALUATE] Coverage check:
  Sources gathered: 3
  Searches done: 1/3
  ❌ Need more searches: INSUFFICIENT...
  → Continuing research (2 search(es) remaining)

[SEARCH 2/3] Query: LangGraph agent orchestration real-world applications limitations

[EVALUATE] Coverage check:
  Sources gathered: 3
  Searches done: 1/3
  ❌ Need more searches: INSUFFICIENT...
  → Continuing research (2

---

### ✅ Part 3 Complete: Plan-Execution Loops

**What You Just Built:**

```mermaid
graph LR
    A[plan: create 3 queries] --> B[search query 1]
    B --> C[evaluate coverage]
    C -->|insufficient| D[search query 2]
    D --> E[evaluate coverage]
    E -->|insufficient| F[search query 3]
    F --> G[evaluate coverage]
    G -->|sufficient| H[synthesize answer]
```

### 🔑 Key Innovation: Different Actions Each Loop!

**Parts 1 & 2**: Same action repeated
```
improve → improve → improve
```

**Part 3**: Plan executed sequentially
```
search "benefits" → search "challenges" → search "comparison"
```

### 🎯 The Four Phases

1. **PLAN Phase** (once)
   - LLM breaks question into 2-3 search angles
   - Stored in `search_plan` list

2. **SEARCH Loop** (repeats)
   - Execute `search_plan[current_search_idx]`
   - Accumulate results
   - Increment `current_search_idx`

3. **EVALUATE** (after each search)
   - Do we have enough information?
   - Early exit if yes! (saves API calls)

4. **SYNTHESIZE** (once at end)
   - Combine all results
   - Cite sources
   - Produce final answer

### 💡 State Management Pattern

```python
class ResearchLoopState:
    search_plan: List[str]         # ["query1", "query2", "query3"]
    current_search_idx: int        # Which search are we on? (0, 1, 2)
    all_results: List[Dict]        # Accumulated results from all searches
    is_sufficient: bool            # Early exit flag
```

**Why this works**:
- `search_plan` = **the roadmap**
- `current_search_idx` = **where we are** on the roadmap
- `all_results` = **what we've collected** so far

### 🚀 Real-World Applications

✅ **Research agents**: Comprehensive information gathering  
✅ **Competitive analysis**: Search multiple competitors  
✅ **Due diligence**: Investigate different aspects  
✅ **Multi-source fact-checking**: Verify from different angles  
✅ **Market research**: Gather diverse perspectives  

### 💰 Why Early Exit Matters

```python
# Without early exit: Always 3 searches = 3 API calls
search 1 → search 2 → search 3 → synthesize

# With early exit: Stop when sufficient
search 1 → search 2 → sufficient! → synthesize  (saved 1 call!)
```

**In production**: This saves money and latency!

### 🎓 Progression So Far

| Part | Loop Type | Exit Strategy |
|------|-----------|---------------|
| Part 1 | Fixed iteration | Counter (3 loops) |
| Part 2 | Quality-driven | Score ≥ 80 |
| Part 3 | Plan-execution | Plan complete OR sufficient |

**Pattern**: Each part builds on the previous! 🎯

### 🧪 Experiments to Try

1. **Modify plan**: Change `plan_searches()` to create 4-5 queries instead of 2-3
2. **Adjust threshold**: Make `evaluate_coverage()` more/less strict
3. **Different domain**: Try medical research, legal research, technical docs

---

**Next**: Now that you master the core patterns, let's combine loops with advanced features like **streaming** and **persistence**! 👉

---

## 🎬 Part 4: Advanced Features — Streaming & Persistent Loops

**You've mastered loops! Now let's add production features.**

### Feature 1: Streaming 🔴 LIVE

**Problem**: Your quality loop runs 5 iterations. You wait... and wait... then finally see results.

**Better**: Watch each iteration **in real-time** as it happens!

```python
for update in agent.stream(state):
    print(f"Iteration {update['iteration']}: Score {update['score']}/100")
```

### Feature 2: Checkpointing 💾

**Problem**: Your research loop takes 3 minutes. Browser crashes. Start over. 😢

**Better**: Save state after each iteration. Resume right where you left off!

```python
# Run iteration 1-2
agent.invoke(state, config={"thread_id": "research_123"})
# Browser crashes...
# Resume from iteration 3
agent.invoke(state, config={"thread_id": "research_123"})  # Continues!
```

### What You'll Learn

✅ Real-time loop monitoring with `.stream()`  
✅ Persistent state with `SqliteSaver`  
✅ Thread-based conversations  
✅ Resume interrupted workflows  

**Let's add these superpowers!** 👇

In [ ]:
# Setup persistent checkpointer
STATE_DIR = Path.cwd() / "langgraph_state"
STATE_DIR.mkdir(parents=True, exist_ok=True)
DB_PATH = STATE_DIR / "workshop_v4_loops.db"

try:
    conn = sqlite3.connect(str(DB_PATH), check_same_thread=False)
    checkpointer = SqliteSaver(conn)
    atexit.register(conn.close)
    print(f"✅ Checkpointer ready at {DB_PATH}")
except Exception as exc:
    conn = sqlite3.connect(":memory:", check_same_thread=False)
    checkpointer = SqliteSaver(conn)
    print(f"⚠️ Using in-memory checkpointer")

# Compile reflection agent with persistence
persistent_reflection = reflection_loop.compile(checkpointer=checkpointer)

In [ ]:
def stream_reflection(query: str, thread_id: str = "default"):
    """Stream each iteration of the reflection loop."""
    print(f"\n{'='*70}")
    print(f"STREAMING REFLECTION LOOP: {query}")
    print(f"{'='*70}\n")
    
    config = {"configurable": {"thread_id": thread_id}}
    state = ReflectionState(query=query)
    
    for update in persistent_reflection.stream(state, config):
        for node_name, delta in update.items():
            print(f"[STREAM] node={node_name:>10} | iteration={delta.get('iteration', '?')}")
    
    print(f"\n{'='*70}")
    print("STREAM COMPLETE")
    print(f"{'='*70}\n")


# Uncomment to watch streaming
# stream_reflection("How do LangGraph loops improve agent reliability?")

---

## 🤝 Part 5: Multi-Agent Debate Loop — Collaborative Refinement

**Final pattern**: What if we loop between **two different agents**?

### The Challenge

Agent A (Proposer): "Here's my answer!"  
Agent B (Critic): "Not good enough, improve X and Y"  
Agent A: "How about now?"  
Agent B: "Perfect! I agree!" ✅

**This is collaborative refinement through debate!**

### Why This Matters

Real-world scenarios:
- **Code review**: Author → Reviewer → Author → Approve
- **Proposal refinement**: Drafter → Stakeholder feedback → Revision
- **Adversarial validation**: Generator → Discriminator (like GANs!)

### What You'll Learn

✅ Multiple agents in one workflow  
✅ Consensus detection as exit condition  
✅ Agent-to-agent communication patterns  
✅ When collaboration beats solo work  

**Let's build a debate system!** 👇

In [ ]:
class DebateState(BaseModel):
    messages: Annotated[List[Union[HumanMessage, AIMessage]], add_messages] = Field(default_factory=list)
    query: str = ""
    proposal_text: str = ""  # Renamed from 'proposal' to avoid node name conflict
    critique_text: str = ""  # Renamed from 'critique_response'
    round: int = 0
    max_rounds: int = 2
    consensus: bool = False
    
    class Config:
        arbitrary_types_allowed = True


def proposer_node(state: DebateState):  # Renamed function for clarity
    """Agent A proposes or refines answer."""
    round_num = state.round + 1
    
    if round_num == 1:
        prompt = f"Propose an answer to: {state.query}"
    else:
        prompt = f"Refine your answer based on critique:\n\nQuery: {state.query}\nPrevious: {state.proposal_text}\nCritique: {state.critique_text}\n\nImproved answer:"
    
    proposal = llm.invoke(prompt).content
    print(f"\n[PROPOSER Round {round_num}]")
    print(f"  {proposal[:120]}...")
    
    return {"proposal_text": proposal, "round": round_num, "messages": [AIMessage(content=f"Proposer: {proposal}", name="proposer")]}


def critic_node(state: DebateState):  # Renamed function for clarity
    """Agent B critiques proposal."""
    prompt = f"""Critique this answer:\n\nQuery: {state.query}\nProposal: {state.proposal_text}\n\nIf the answer is good enough, start your response with "AGREE".
Otherwise, suggest specific improvements."""
    
    critique = llm.invoke(prompt).content.strip()
    # More robust consensus detection
    consensus = critique.upper().startswith("AGREE")
    
    print(f"[CRITIC Round {state.round}]")
    if consensus:
        print(f"  ✅ CONSENSUS reached!")
    else:
        print(f"  💬 Feedback: {critique[:100]}...")
    
    return {"critique_text": critique, "consensus": consensus, "messages": [AIMessage(content=f"Critic: {critique}", name="critic")]}


def should_debate_continue(state: DebateState) -> str:
    if state.consensus:
        print(f"  → Consensus! Ending debate.")
        return "end"
    elif state.round >= state.max_rounds:
        print(f"  → Max rounds reached. Accepting proposal.")
        return "end"
    else:
        print(f"  → Continuing debate (round {state.round + 1})")
        return "continue"


# Build debate loop
debate_loop = StateGraph(DebateState)
debate_loop.add_node("proposer", proposer_node)  # Node name: "proposer" (no conflict!)
debate_loop.add_node("critic", critic_node)      # Node name: "critic" (no conflict!)

debate_loop.set_entry_point("proposer")
debate_loop.add_edge("proposer", "critic")

# THE LOOP: debate continues until consensus or max rounds
debate_loop.add_conditional_edges(
    "critic",
    should_debate_continue,
    {
        "continue": "proposer",  # 🔄 Loop back to proposer
        "end": END
    }
)

debate_agent = debate_loop.compile()
print("✅ Debate agent compiled")

In [ ]:
# Try it: Watch two agents debate
query = "What makes loops essential for production AI agents?"
result = debate_agent.invoke(DebateState(query=query))

print(f"\n{'='*70}")
print(f"DEBATE COMPLETE after {result['round']} round(s)")
print(f"{'='*70}")
print(f"\nFinal proposal:\n{result['proposal_text']}")
print(f"\nFinal critique:\n{result['critique_text']}")

---

### ✅ Part 5 Complete: Multi-Agent Loops

**What You Just Built:**

```mermaid
graph LR
    A[proposer: draft] --> B[critic: review]
    B -->|disagree| A
    B -->|agree| C[END]
```

### 🔑 Key Pattern: Agent Ping-Pong

**Different from Parts 1-3**: Instead of ONE agent improving itself, TWO agents work together!

```python
# Part 2: Single agent self-critiques
generate → critique (same LLM) → generate

# Part 5: Two agents collaborate  
proposer → critic (different perspective) → proposer
```

### 💡 Exit Condition: Consensus Detection

```python
def critic_node(state):
    critique = llm.invoke("Review this...")
    consensus = critique.upper().startswith("AGREE")  # Keyword detection
    return {"consensus": consensus, ...}

def should_debate_continue(state):
    if state.consensus:
        return "end"  # ✅ Both agree!
    elif state.round >= max_rounds:
        return "end"  # ⚠️ Time's up, accept proposal
    else:
        return "continue"  # 🔄 Keep debating
```

### 🎯 When to Use Multi-Agent Loops

✅ **Excellent for**:
- **Code review workflows**: Author ↔ Reviewer
- **Proposal refinement**: Drafter ↔ Stakeholder
- **Adversarial testing**: Generator ↔ Validator
- **Roleplay scenarios**: Different perspectives improve output
- **Expert panel**: Multiple specialized agents discuss

❌ **Overkill for**:
- Simple tasks (single agent sufficient)
- High-latency scenarios (2× LLM calls per round)
- When consensus isn't needed

### 🧠 Advanced Patterns

**What we built**: 2 agents, simple consensus

**Production patterns**:
- **3+ agents**: Panel of experts vote
- **Weighted consensus**: Senior agent has veto power
- **Hierarchical**: Manager → Team → Manager approval
- **Adversarial**: Red team vs Blue team

### 🆚 Compare All Patterns

| Part | Agents | Loop Target | Exit |
|------|--------|-------------|------|
| 1 | 1 agent | Same action 3× | Counter |
| 2 | 1 agent | Self-improve | Score ≥ 80 |
| 3 | 1 agent | Different searches | Plan done |
| 5 | 2 agents | Debate rounds | Consensus |

**The power**: Mix and match these patterns! 🎨

---

**Final step**: Let's wrap up everything you've learned! 👇

---

# ? Workshop Complete: You're Now a Loop Master!

Congratulations! You've built 5 different loop patterns from scratch. Let's solidify what you've learned.

---

## 📚 Quick Reference: Your Loop Toolkit

### The 5 Patterns You Mastered

| Pattern | When to Use | Exit Strategy | Complexity |
|---------|-------------|---------------|------------|
| **🔁 Fixed Iteration** | You know exactly how many rounds needed | `iteration >= N` | ⭐ Beginner |
| **📊 Quality-Driven** | Loop until output meets standards | `score >= threshold` + max | ⭐⭐ Intermediate |
| **🔍 Plan-Execution** | Execute list of different actions | Plan complete + early exit | ⭐⭐ Intermediate |
| **🎬 Streaming/Persistent** | Long-running, resumable workflows | Any of above + checkpointing | ⭐⭐⭐ Advanced |
| **🤝 Multi-Agent** | Collaborative refinement needed | Consensus + max rounds | ⭐⭐⭐ Advanced |

---

## 🎯 Decision Tree: Which Loop Should I Use?

```
Do you know the exact number of iterations needed?
├─ YES → Fixed Iteration Loop (Part 1)
└─ NO → Continue...
    
    Are you executing different actions each loop?
    ├─ YES → Plan-Execution Loop (Part 3)
    └─ NO → Continue...
        
        Is quality measurable with a score?
        ├─ YES → Quality-Driven Loop (Part 2)
        └─ NO → Continue...
            
            Do you need multiple perspectives?
            ├─ YES → Multi-Agent Loop (Part 5)
            └─ NO → Maybe you don't need a loop! 🤔
```

---

## ⚡ The Golden Rules of Loop Design

### ✅ Always Do This

1. **Include `max_iterations`** - NEVER ship without a safety limit!
   ```python
   max_iterations: int = 5  # Prevent infinite loops
   ```

2. **Track iterations in state** - Essential for debugging
   ```python
   iteration: int = 0  # Always increment this
   ```

3. **Log progress** - Help users (and yourself) understand what's happening
   ```python
   print(f"[Iteration {iteration}] Score: {score}/100")
   ```

4. **Have TWO exit conditions** - Quality goal + safety limit
   ```python
   if score >= 80:      # ✅ Goal achieved
       return "done"
   elif iteration >= 5: # ⚠️ Safety limit
       return "done"
   ```

### ⚠️ Common Pitfalls to Avoid

| Mistake | Consequence | Solution |
|---------|-------------|----------|
| No `max_iterations` | Infinite loop 🔥 | Always set a limit! |
| Too strict quality check | Always hits max iterations | Use scoring (60-100), not binary |
| Too lenient quality check | Exits too early, poor results | Tune threshold with real data |
| Not tracking cost | $$$ explosion | Monitor LLM call count |
| Forgetting state evolution | Loops don't improve | Carry feedback between iterations |

---

## 🔧 Production Patterns: Combining Loops

### Pattern 1: Quality Loop with Retry Logic
```python
# Outer loop: Quality-driven refinement
# Inner loop: Retry each LLM call with exponential backoff

generate (with retry) → critique → refine (with retry) → repeat
```

### Pattern 2: Multi-Agent + Streaming
```python
# Watch debate in real-time
for update in debate_agent.stream(state):
    print(f"Round {update['round']}: {update['proposal_text'][:50]}...")
```

### Pattern 3: Research + Validation (Nested Loops)
```python
# Outer: Validate research quality
# Inner: Execute search plan

validate → [insufficient? → replan → search (loop) → synthesize] → repeat
```

---

## 🚀 Next Steps: Level Up Your Skills

### 🎯 Immediate Practice (Do This Today!)

1. **Modify Part 2**: Change scoring to use YOUR quality criteria
2. **Extend Part 3**: Add a 4th search to the plan
3. **Combine Parts 2+4**: Add streaming to the quality loop

### 📈 Intermediate Projects (This Week)

1. **Code Review Agent**: 
   - Part 2 (quality) + Part 5 (author ↔ reviewer)
   
2. **Research Assistant**:
   - Part 3 (multi-search) + Part 2 (validate answer quality)
   
3. **Content Generator**:
   - Part 1 (generate N drafts) → Part 2 (pick best and refine)

### 🏆 Advanced Challenges (When Ready)

1. **Nested Loops**: Combine Part 2 inside Part 3
2. **Parallel Loops**: Run multiple Part 1 loops simultaneously
3. **HITL Loops**: Add `interrupt()` for human approval
4. **Adaptive Thresholds**: Learn optimal `score_threshold` over time

---

## 📖 Key Takeaway

### Before This Workshop:
```python
# One-shot agent (brittle!)
user_question → llm.invoke() → answer
```

### After This Workshop:
```python
# Self-improving agent (resilient!)
user_question → [generate → evaluate → refine] × N → quality_answer
```

**You've transformed brittle one-shot agents into resilient, self-improving systems!** 🎉

---

## 🎓 Certification: You Are Now

✅ Able to build loops that prevent infinite cycles  
✅ Skilled in quality-driven iteration  
✅ Confident with plan-execution patterns  
✅ Ready to combine loops for production systems  
✅ Aware of cost/quality tradeoffs  

**Welcome to the world of production-ready AI agents!** 🚀

---

## 📚 Further Learning

- **LangGraph Docs**: [langgraph.langchain.com](https://langchain-ai.github.io/langgraph/)
- **Advanced Patterns**: See `Workshop_agent_v4_advanced.ipynb` for nested loops
- **Community**: Share your loop patterns with the community!

---

**Final thought**: The difference between a demo and a production agent is often just one thing: **loops**. You now have that superpower! 💪

**Thank you for attending this workshop!** ?